#### (1) Data collection using both, Web Scraping and a Web API.
#### (2) Data preparation (e.g. remove missing values and duplicates, create new variables, enrich the data with open data).
#### (3) Data storage in a database like sqlite or MySQL.
#### (4) Non-graphical and graphical exploratory data analysis (EDA).
#### (5) Use of either regression or classification as the modelling method.
#### (6) Model evaluation using suitable measures of fit.
#### (7) Correct interpretation of model results and measures of fit.
#### (8) Making the material (data, Jupyter notebooks, ...) available on Moodle.

## Overpass turbo query to get all available supermarkets in Switzerland

In [ ]:
from pandas import json_normalize
import requests
import json

# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["shop"="supermarket"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('supermarkets.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)
df.head(5)

## Plot supermarkets on map

In [ ]:
import folium
import pandas as pd

# Subset of supermarkets by brand
locations = df[["lat", "lon", "tags.brand", "tags.shop"]].loc[df["tags.brand"] == 'Migros']
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.brand"]).add_to(map)

# Plot map
map